In [1]:
import tensorflow as tf# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# instantiate a distribution strategy
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

E0000 00:00:1734599076.831208      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D1219 09:04:36.839379540      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1219 09:04:36.839393512      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1219 09:04:36.839396922      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1219 09:04:36.839399492      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1219 09:04:36.839401915      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1734599089.861581      13 service.cc:145] XLA service 0x5845b612adb0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734599089.861633      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1734599089.861637      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1734599089.861640      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1734599089.861643      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1734599089.861646      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1734599089.861649      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1734599089.861651      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1734599089.861654      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [2]:
pip install scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 50.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.4/315.4 kB 23.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import cv2
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from skimage import img_as_float
import matplotlib.pyplot as plt
with tpu_strategy.scope():
    # Custom simplified NIQE computation
    def compute_niqe_simplified(image):
        """Simplified approximation of NIQE."""
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        mean = np.mean(image_gray)
        std = np.std(image_gray)
        return mean / (std + 1e-5)  # Simplified NIQE metric
    
    # Compute SSIM with proper handling of window size
    def compute_ssim(img1, img2):
        min_dim = min(img1.shape[:2])
        win_size = min(7, min_dim)
        return ssim(img1, img2, data_range=255, channel_axis=-1, win_size=win_size)
    
    # Compute Sharpness (Laplacian variance)
    def compute_sharpness(image):
        return cv2.Laplacian(image, cv2.CV_64F).var()
    
    # Process and compare two videos
    def compare_videos(input_video_path_low_quality, input_video_path_enhanced):
        # Open both videos
        cap_low_quality = cv2.VideoCapture(input_video_path_low_quality)
        cap_enhanced = cv2.VideoCapture(input_video_path_enhanced)
    
        # Get video properties (assuming both videos have same properties)
        fps = int(cap_low_quality.get(cv2.CAP_PROP_FPS))
        width = int(cap_low_quality.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap_low_quality.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap_low_quality.get(cv2.CAP_PROP_FRAME_COUNT))
    
        # Initialize lists to store metrics
        psnr_values_low, psnr_values_enhanced = [], []
        ssim_values_low, ssim_values_enhanced = [], []
        sharpness_values_low, sharpness_values_enhanced = [], []
        niqe_values_low, niqe_values_enhanced = [], []
    
        frame_idx = 0
        while cap_low_quality.isOpened() and cap_enhanced.isOpened():
            ret_low, frame_low = cap_low_quality.read()
            ret_enhanced, frame_enh = cap_enhanced.read()
    
            if not ret_low or not ret_enhanced:
                break
    
            frame_idx += 1
            print(f"Processing frame {frame_idx}/{total_frames}", end="\r")
    
            # Ensure the frames are in uint8 format
            frame_low = (frame_low * 255).astype(np.uint8) if np.max(frame_low) <= 1.0 else frame_low.astype(np.uint8)
            frame_enh = (frame_enh * 255).astype(np.uint8) if np.max(frame_enh) <= 1.0 else frame_enh.astype(np.uint8)
    
            # Compute PSNR, SSIM, Sharpness, and NIQE for both videos
            psnr_values_low.append(psnr(frame_low, frame_enh, data_range=255))
            ssim_values_low.append(compute_ssim(frame_low, frame_enh))
            sharpness_values_low.append(compute_sharpness(frame_low))
            niqe_values_low.append(compute_niqe_simplified(frame_low))
    
            psnr_values_enhanced.append(psnr(frame_enh, frame_low, data_range=255))
            ssim_values_enhanced.append(compute_ssim(frame_enh, frame_low))
            sharpness_values_enhanced.append(compute_sharpness(frame_enh))
            niqe_values_enhanced.append(compute_niqe_simplified(frame_enh))
    
        # Release resources
        cap_low_quality.release()
        cap_enhanced.release()
    
        # Compute average metrics for each video
        avg_psnr_low = np.mean(psnr_values_low)
        avg_psnr_enhanced = np.mean(psnr_values_enhanced)
        avg_ssim_low = np.mean(ssim_values_low)
        avg_ssim_enhanced = np.mean(ssim_values_enhanced)
        avg_sharpness_low = np.mean(sharpness_values_low)
        avg_sharpness_enhanced = np.mean(sharpness_values_enhanced)
        avg_niqe_low = np.mean(niqe_values_low)
        avg_niqe_enhanced = np.mean(niqe_values_enhanced)
    
        print("\nProcessing completed.")
        print("Low-Quality Video:")
        print("Average PSNR:", avg_psnr_low)
        print("Average SSIM:", avg_ssim_low)
        print("Average Sharpness (Low-Quality):", avg_sharpness_low)
        print("Average NIQE (Low-Quality):", avg_niqe_low)
    
        print("\nEnhanced Video:")
        print("Average PSNR:", avg_psnr_enhanced)
        print("Average SSIM:", avg_ssim_enhanced)
        print("Average Sharpness (Enhanced):", avg_sharpness_enhanced)
        print("Average NIQE (Enhanced):", avg_niqe_enhanced)




In [4]:
# Example Usage
with tpu_strategy.scope():
    input_video_path_low_quality = '/kaggle/input/new2-enhancement/000.mp4'
    input_video_path_enhanced = '/kaggle/input/new2-enhancement/000_c23.mp4'
    
    compare_videos(input_video_path_low_quality, input_video_path_enhanced)

Processing frame 396/396
Processing completed.
Low-Quality Video:
Average PSNR: 31.886113974939313
Average SSIM: 0.9160713974986188
Average Sharpness (Low-Quality): 147.71665615292864
Average NIQE (Low-Quality): 2.5310765438046423

Enhanced Video:
Average PSNR: 31.886113974939313
Average SSIM: 0.9160713974986188
Average Sharpness (Enhanced): 215.62159953522095
Average NIQE (Enhanced): 2.4951498123088456


In [5]:
with tpu_strategy.scope():
    input_video_path_low_quality = '/kaggle/input/new2-enhancement/023.mp4'
    input_video_path_enhanced = '/kaggle/input/new2-enhancement/023_c23_enhanced_video_new.mp4'
    
    compare_videos(input_video_path_low_quality, input_video_path_enhanced)


Processing frame 553/553
Processing completed.
Low-Quality Video:
Average PSNR: 25.676724088000217
Average SSIM: 0.8314965204540236
Average Sharpness (Low-Quality): 141.73033513299367
Average NIQE (Low-Quality): 1.488044624354714

Enhanced Video:
Average PSNR: 25.676724088000217
Average SSIM: 0.8314965204540236
Average Sharpness (Enhanced): 668.7435729701459
Average NIQE (Enhanced): 1.3608145553626192


In [6]:
with tpu_strategy.scope():
    input_video_path_low_quality = '/kaggle/input/new2-enhancement/459.mp4'
    input_video_path_enhanced = '/kaggle/input/new2-enhancement/459_enhanced_video_new.mp4'
    
    compare_videos(input_video_path_low_quality, input_video_path_enhanced)


Processing frame 526/526
Processing completed.
Low-Quality Video:
Average PSNR: 35.10278572379885
Average SSIM: 0.9385959945255533
Average Sharpness (Low-Quality): 11.436255524528173
Average NIQE (Low-Quality): 3.2197726293032884

Enhanced Video:
Average PSNR: 35.10278572379885
Average SSIM: 0.9385959945255533
Average Sharpness (Enhanced): 43.01036867758626
Average NIQE (Enhanced): 3.1698124620349546


In [ ]:
with tpu_strategy.scope():# Example Usage
    input_video_path_low_quality = '/kaggle/input/new2-enhancement/459.mp4'
    input_video_path_enhanced = '/kaggle/input/new2-enhancement/459_c23_enhanced_video_new.mp4'
    
    compare_videos(input_video_path_low_quality, input_video_path_enhanced)

In [8]:
# Example Usage
with tpu_strategy.scope():
    input_video_path_low_quality = '/kaggle/input/new2-enhancement/818.mp4'
    input_video_path_enhanced = '/kaggle/input/new2-enhancement/818_enhanced_video_new.mp4'
    
    compare_videos(input_video_path_low_quality, input_video_path_enhanced)

KeyboardInterrupt: 

In [ ]:
# Example Usage
with tpu_strategy.scope():
    input_video_path_low_quality = '/kaggle/input/new2-enhancement/818.mp4'
    input_video_path_enhanced = '/kaggle/input/new2-enhancement/818_c23_enhanced_video_new.mp4'
    
    compare_videos(input_video_path_low_quality, input_video_path_enhanced)

------

In [3]:
import cv2
import numpy as np
with tpu_strategy.scope():
    # Read input video
    cap = cv2.VideoCapture('/kaggle/input/c23-dataset/459_c23.mp4')
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('/kaggle/working/459_c23_enhanced_video_new.mp4', fourcc, 30.0, (width, height))
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")
        # Denoise and sharpen the frame
        denoised = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
        sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened = cv2.filter2D(denoised, -1, sharpen_kernel)
    
        out.write(sharpened)
    
    cap.release()
    out.release()


In [4]:
import cv2
import numpy as np
with tpu_strategy.scope():
    # Read input video
    cap = cv2.VideoCapture('/kaggle/input/c23-dataset/023_c23.mp4')
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('/kaggle/working/023_c23_enhanced_video_new.mp4', fourcc, 30.0, (width, height))
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")
        # Denoise and sharpen the frame
        denoised = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
        sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened = cv2.filter2D(denoised, -1, sharpen_kernel)
    
        out.write(sharpened)
    
    cap.release()
    out.release()


In [5]:
import cv2
import numpy as np
with tpu_strategy.scope():
    # Read input video
    cap = cv2.VideoCapture('/kaggle/input/c23-dataset/818_c23.mp4')
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('/kaggle/working/818_c23_enhanced_video_new.mp4', fourcc, 30.0, (width, height))
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")
        # Denoise and sharpen the frame
        denoised = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
        sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened = cv2.filter2D(denoised, -1, sharpen_kernel)
    
        out.write(sharpened)
    
    cap.release()
    out.release()


-------

In [5]:
import cv2
import numpy as np

# Read input video
cap = cv2.VideoCapture('/kaggle/input/video-metrics/459.mp4')

# Retrieve the video frame width and height
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/kaggle/working/459_enhanced_video_new.mp4', fourcc, 30.0, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Denoise and sharpen the frame
    denoised = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
    sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(denoised, -1, sharpen_kernel)

    out.write(sharpened)

cap.release()
out.release()


In [7]:
import cv2

# Read input video
cap = cv2.VideoCapture('/kaggle/input/video-metrics2/023.mp4')

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/kaggle/working/023_enhanced_video_new.mp4', fourcc, 30.0, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Denoise and sharpen the frame
    denoised = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
    sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(denoised, -1, sharpen_kernel)

    out.write(sharpened)
    cv2.imshow('Frame', sharpened)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
with tpu_strategy.scope():
    # Read input video
    cap = cv2.VideoCapture('/kaggle/input/video-metrics/102.mp4')
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('/kaggle/working/102_enhanced_video_new.mp4', fourcc, 30.0, (width, height))
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")
        # Denoise and sharpen the frame
        denoised = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
        sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened = cv2.filter2D(denoised, -1, sharpen_kernel)
    
        out.write(sharpened)
    
    cap.release()
    out.release()


In [2]:
import cv2
import numpy as np
with tpu_strategy.scope():
    # Read input video
    cap = cv2.VideoCapture('/kaggle/input/video-metrics/500.mp4')
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('/kaggle/working/500_enhanced_video_new.mp4', fourcc, 30.0, (width, height))
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")
        # Denoise and sharpen the frame
        denoised = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
        sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened = cv2.filter2D(denoised, -1, sharpen_kernel)
    
        out.write(sharpened)
    
    cap.release()
    out.release()


In [3]:
import cv2
import numpy as np
with tpu_strategy.scope():
    # Read input video
    cap = cv2.VideoCapture('/kaggle/input/video-metrics/477_487.mp4')
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('/kaggle/working/477_487_enhanced_video_new.mp4', fourcc, 30.0, (width, height))
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")
        # Denoise and sharpen the frame
        denoised = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
        sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened = cv2.filter2D(denoised, -1, sharpen_kernel)
    
        out.write(sharpened)
    
    cap.release()
    out.release()


In [5]:
import cv2
import numpy as np
with tpu_strategy.scope():
    # Read input video
    cap = cv2.VideoCapture('/kaggle/input/video-metrics/818.mp4')
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('/kaggle/working/818_enhanced_video_new.mp4', fourcc, 30.0, (width, height))
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")
        # Denoise and sharpen the frame
        denoised = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
        sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        sharpened = cv2.filter2D(denoised, -1, sharpen_kernel)
    
        out.write(sharpened)
    
    cap.release()
    out.release()


-----------------------------------

In [2]:
import numpy as np
import cv2
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from skimage import img_as_float
import matplotlib.pyplot as plt

# Custom simplified NIQE computation
def compute_niqe_simplified(image):
    """Simplified approximation of NIQE."""
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean = np.mean(image_gray)
    std = np.std(image_gray)
    return mean / (std + 1e-5)  # Simplified NIQE metric

# Image Enhancement: Histogram Equalization on each channel
def enhance_image(image):
    if image.max() <= 1.0:
        image = (image * 255).astype(np.uint8)

    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = l.astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_enhanced = clahe.apply(l)
    lab_enhanced = cv2.merge((l_enhanced, a, b))
    enhanced_image = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
    return enhanced_image

# Compute SSIM with proper handling of window size
def compute_ssim(img1, img2):
    min_dim = min(img1.shape[:2])
    win_size = min(7, min_dim)
    return ssim(img1, img2, data_range=255, channel_axis=-1, win_size=win_size)

# Process Video
def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Video writer for enhanced video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    psnr_values, ssim_values, sharpness_original, sharpness_enhanced = [], [], [], []
    niqe_original, niqe_enhanced = [], []

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")

        # Enhance frame
        enhanced_frame = enhance_image(frame)

        # Normalize frames to uint8 if needed
        frame = (frame * 255).astype(np.uint8) if np.max(frame) <= 1.0 else frame.astype(np.uint8)
        enhanced_frame = (enhanced_frame * 255).astype(np.uint8) if np.max(enhanced_frame) <= 1.0 else enhanced_frame.astype(np.uint8)

        # Compute metrics
        psnr_values.append(psnr(frame, enhanced_frame, data_range=255))
        ssim_values.append(compute_ssim(frame, enhanced_frame))
        sharpness_original.append(cv2.Laplacian(frame, cv2.CV_64F).var())
        sharpness_enhanced.append(cv2.Laplacian(enhanced_frame, cv2.CV_64F).var())
        niqe_original.append(compute_niqe_simplified(frame))
        niqe_enhanced.append(compute_niqe_simplified(enhanced_frame))

        # Write enhanced frame to output video
        out.write(enhanced_frame)

    # Release resources
    cap.release()
    out.release()

    # Compute average metrics
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)
    avg_sharpness_original = np.mean(sharpness_original)
    avg_sharpness_enhanced = np.mean(sharpness_enhanced)
    avg_niqe_original = np.mean(niqe_original)
    avg_niqe_enhanced = np.mean(niqe_enhanced)

    print("\nProcessing completed.")
    print("Average PSNR:", avg_psnr)
    print("Average SSIM:", avg_ssim)
    print("Average Sharpness (Original):", avg_sharpness_original)
    print("Average Sharpness (Enhanced):", avg_sharpness_enhanced)
    print("Average NIQE (Original):", avg_niqe_original)
    print("Average NIQE (Enhanced):", avg_niqe_enhanced)

# Example Usage
input_video_path = '/kaggle/input/video-metrics/039_058.mp4'
output_video_path = '/kaggle/working/039_058_enhanced_video.mp4'
process_video(input_video_path, output_video_path)


Processing frame 609/609
Processing completed.
Average PSNR: 19.99177283293613
Average SSIM: 0.8530796320636583
Average Sharpness (Original): 675.381643535497
Average Sharpness (Enhanced): 989.8783054486905
Average NIQE (Original): 1.7072791677495633
Average NIQE (Enhanced): 1.768468701560128


In [3]:
import numpy as np
import cv2
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from skimage import img_as_float
import matplotlib.pyplot as plt

# Custom simplified NIQE computation
def compute_niqe_simplified(image):
    """Simplified approximation of NIQE."""
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean = np.mean(image_gray)
    std = np.std(image_gray)
    return mean / (std + 1e-5)  # Simplified NIQE metric

# Image Enhancement: Histogram Equalization on each channel
def enhance_image(image):
    if image.max() <= 1.0:
        image = (image * 255).astype(np.uint8)

    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = l.astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_enhanced = clahe.apply(l)
    lab_enhanced = cv2.merge((l_enhanced, a, b))
    enhanced_image = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
    return enhanced_image

# Compute SSIM with proper handling of window size
def compute_ssim(img1, img2):
    min_dim = min(img1.shape[:2])
    win_size = min(7, min_dim)
    return ssim(img1, img2, data_range=255, channel_axis=-1, win_size=win_size)

# Process Video
def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Video writer for enhanced video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    psnr_values, ssim_values, sharpness_original, sharpness_enhanced = [], [], [], []
    niqe_original, niqe_enhanced = [], []

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")

        # Enhance frame
        enhanced_frame = enhance_image(frame)

        # Normalize frames to uint8 if needed
        frame = (frame * 255).astype(np.uint8) if np.max(frame) <= 1.0 else frame.astype(np.uint8)
        enhanced_frame = (enhanced_frame * 255).astype(np.uint8) if np.max(enhanced_frame) <= 1.0 else enhanced_frame.astype(np.uint8)

        # Compute metrics
        psnr_values.append(psnr(frame, enhanced_frame, data_range=255))
        ssim_values.append(compute_ssim(frame, enhanced_frame))
        sharpness_original.append(cv2.Laplacian(frame, cv2.CV_64F).var())
        sharpness_enhanced.append(cv2.Laplacian(enhanced_frame, cv2.CV_64F).var())
        niqe_original.append(compute_niqe_simplified(frame))
        niqe_enhanced.append(compute_niqe_simplified(enhanced_frame))

        # Write enhanced frame to output video
        out.write(enhanced_frame)

    # Release resources
    cap.release()
    out.release()

    # Compute average metrics
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)
    avg_sharpness_original = np.mean(sharpness_original)
    avg_sharpness_enhanced = np.mean(sharpness_enhanced)
    avg_niqe_original = np.mean(niqe_original)
    avg_niqe_enhanced = np.mean(niqe_enhanced)

    print("\nProcessing completed.")
    print("Average PSNR:", avg_psnr)
    print("Average SSIM:", avg_ssim)
    print("Average Sharpness (Original):", avg_sharpness_original)
    print("Average Sharpness (Enhanced):", avg_sharpness_enhanced)
    print("Average NIQE (Original):", avg_niqe_original)
    print("Average NIQE (Enhanced):", avg_niqe_enhanced)

# Example Usage
input_video_path = '/kaggle/input/video-metrics2/023.mp4'
output_video_path = '/kaggle/working/023_enhanced_video.mp4'
process_video(input_video_path, output_video_path)


Processing frame 553/553
Processing completed.
Average PSNR: 18.03250120010604
Average SSIM: 0.7956313399386197
Average Sharpness (Original): 141.73033513299367
Average Sharpness (Enhanced): 358.27596305674274
Average NIQE (Original): 1.488044624354714
Average NIQE (Enhanced): 1.583206514414064


In [9]:
# Example Usage
input_video_path = '/kaggle/input/video-metrics2/025_067.mp4'
output_video_path = '/kaggle/working/025_067_enhanced_video.mp4'
process_video(input_video_path, output_video_path)


KeyboardInterrupt: 

In [5]:
import numpy as np
import cv2
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from skimage import img_as_float
import matplotlib.pyplot as plt

# Custom simplified NIQE computation
def compute_niqe_simplified(image):
    """Simplified approximation of NIQE."""
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean = np.mean(image_gray)
    std = np.std(image_gray)
    return mean / (std + 1e-5)  # Simplified NIQE metric

# Image Enhancement: Histogram Equalization on each channel
def enhance_image(image):
    if image.max() <= 1.0:
        image = (image * 255).astype(np.uint8)

    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = l.astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_enhanced = clahe.apply(l)
    lab_enhanced = cv2.merge((l_enhanced, a, b))
    enhanced_image = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
    return enhanced_image

# Compute SSIM with proper handling of window size
def compute_ssim(img1, img2):
    min_dim = min(img1.shape[:2])
    win_size = min(7, min_dim)
    return ssim(img1, img2, data_range=255, channel_axis=-1, win_size=win_size)

# Process Video
def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Video writer for enhanced video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    psnr_values, ssim_values, sharpness_original, sharpness_enhanced = [], [], [], []
    niqe_original, niqe_enhanced = [], []

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")

        # Enhance frame
        enhanced_frame = enhance_image(frame)

        # Normalize frames to uint8 if needed
        frame = (frame * 255).astype(np.uint8) if np.max(frame) <= 1.0 else frame.astype(np.uint8)
        enhanced_frame = (enhanced_frame * 255).astype(np.uint8) if np.max(enhanced_frame) <= 1.0 else enhanced_frame.astype(np.uint8)

        # Compute metrics
        psnr_values.append(psnr(frame, enhanced_frame, data_range=255))
        ssim_values.append(compute_ssim(frame, enhanced_frame))
        sharpness_original.append(cv2.Laplacian(frame, cv2.CV_64F).var())
        sharpness_enhanced.append(cv2.Laplacian(enhanced_frame, cv2.CV_64F).var())
        niqe_original.append(compute_niqe_simplified(frame))
        niqe_enhanced.append(compute_niqe_simplified(enhanced_frame))

        # Write enhanced frame to output video
        out.write(enhanced_frame)

    # Release resources
    cap.release()
    out.release()

    # Compute average metrics
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)
    avg_sharpness_original = np.mean(sharpness_original)
    avg_sharpness_enhanced = np.mean(sharpness_enhanced)
    avg_niqe_original = np.mean(niqe_original)
    avg_niqe_enhanced = np.mean(niqe_enhanced)

    print("\nProcessing completed.")
    print("Average PSNR:", avg_psnr)
    print("Average SSIM:", avg_ssim)
    print("Average Sharpness (Original):", avg_sharpness_original)
    print("Average Sharpness (Enhanced):", avg_sharpness_enhanced)
    print("Average NIQE (Original):", avg_niqe_original)
    print("Average NIQE (Enhanced):", avg_niqe_enhanced)

# Example Usage
input_video_path = '/kaggle/input/video-metrics/102.mp4'
output_video_path = '/kaggle/working/102_enhanced_video.mp4'
process_video(input_video_path, output_video_path)


Processing frame 389/389
Processing completed.
Average PSNR: 19.925091682959803
Average SSIM: 0.8028420609908257
Average Sharpness (Original): 473.15127148056854
Average Sharpness (Enhanced): 646.510046346584
Average NIQE (Original): 1.304553232984052
Average NIQE (Enhanced): 1.5461315377316767


In [6]:
import numpy as np
import cv2
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from skimage import img_as_float
import matplotlib.pyplot as plt

# Custom simplified NIQE computation
def compute_niqe_simplified(image):
    """Simplified approximation of NIQE."""
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean = np.mean(image_gray)
    std = np.std(image_gray)
    return mean / (std + 1e-5)  # Simplified NIQE metric

# Image Enhancement: Histogram Equalization on each channel
def enhance_image(image):
    if image.max() <= 1.0:
        image = (image * 255).astype(np.uint8)

    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = l.astype(np.uint8)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_enhanced = clahe.apply(l)
    lab_enhanced = cv2.merge((l_enhanced, a, b))
    enhanced_image = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
    return enhanced_image

# Compute SSIM with proper handling of window size
def compute_ssim(img1, img2):
    min_dim = min(img1.shape[:2])
    win_size = min(7, min_dim)
    return ssim(img1, img2, data_range=255, channel_axis=-1, win_size=win_size)

# Process Video
def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Video writer for enhanced video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    psnr_values, ssim_values, sharpness_original, sharpness_enhanced = [], [], [], []
    niqe_original, niqe_enhanced = [], []

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        print(f"Processing frame {frame_idx}/{total_frames}", end="\r")

        # Enhance frame
        enhanced_frame = enhance_image(frame)

        # Normalize frames to uint8 if needed
        frame = (frame * 255).astype(np.uint8) if np.max(frame) <= 1.0 else frame.astype(np.uint8)
        enhanced_frame = (enhanced_frame * 255).astype(np.uint8) if np.max(enhanced_frame) <= 1.0 else enhanced_frame.astype(np.uint8)

        # Compute metrics
        psnr_values.append(psnr(frame, enhanced_frame, data_range=255))
        ssim_values.append(compute_ssim(frame, enhanced_frame))
        sharpness_original.append(cv2.Laplacian(frame, cv2.CV_64F).var())
        sharpness_enhanced.append(cv2.Laplacian(enhanced_frame, cv2.CV_64F).var())
        niqe_original.append(compute_niqe_simplified(frame))
        niqe_enhanced.append(compute_niqe_simplified(enhanced_frame))

        # Write enhanced frame to output video
        out.write(enhanced_frame)

    # Release resources
    cap.release()
    out.release()

    # Compute average metrics
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)
    avg_sharpness_original = np.mean(sharpness_original)
    avg_sharpness_enhanced = np.mean(sharpness_enhanced)
    avg_niqe_original = np.mean(niqe_original)
    avg_niqe_enhanced = np.mean(niqe_enhanced)

    print("\nProcessing completed.")
    print("Average PSNR:", avg_psnr)
    print("Average SSIM:", avg_ssim)
    print("Average Sharpness (Original):", avg_sharpness_original)
    print("Average Sharpness (Enhanced):", avg_sharpness_enhanced)
    print("Average NIQE (Original):", avg_niqe_original)
    print("Average NIQE (Enhanced):", avg_niqe_enhanced)

# Example Usage
input_video_path = '/kaggle/input/video-metrics/477_487.mp4'
output_video_path = '/kaggle/working/477_487_enhanced_video.mp4'
process_video(input_video_path, output_video_path)


Processing frame 365/365
Processing completed.
Average PSNR: 15.792193220110514
Average SSIM: 0.7342522921895915
Average Sharpness (Original): 35.44776589499353
Average Sharpness (Enhanced): 145.57336882031032
Average NIQE (Original): 1.7449234040782386
Average NIQE (Enhanced): 1.8530984364118936


In [ ]:
# Example Usage
input_video_path = '/kaggle/input/video-metrics/459.mp4'
output_video_path = '/kaggle/working/459_enhanced_video.mp4'
process_video(input_video_path, output_video_path)


In [8]:

# Example Usage
input_video_path = '/kaggle/input/video-metrics/500.mp4'
output_video_path = '/kaggle/working/500_enhanced_video.mp4'
process_video(input_video_path, output_video_path)


Processing frame 292/292
Processing completed.
Average PSNR: 18.155197303813367
Average SSIM: 0.8454666387847598
Average Sharpness (Original): 127.14719496563379
Average Sharpness (Enhanced): 147.3465092644091
Average NIQE (Original): 1.1846784586035735
Average NIQE (Enhanced): 1.4720926506660685


In [10]:
# Example Usage
input_video_path = '/kaggle/input/video-metrics/576_155.mp4'
output_video_path = '/kaggle/working/576_155_enhanced_video.mp4'
process_video(input_video_path, output_video_path)


Processing frame 461/461
Processing completed.
Average PSNR: 20.135472039571024
Average SSIM: 0.877855489970095
Average Sharpness (Original): 327.79896585445863
Average Sharpness (Enhanced): 375.1489025011959
Average NIQE (Original): 1.7735371309847419
Average NIQE (Enhanced): 1.9937265131858282


In [11]:
# Example Usage
input_video_path = '/kaggle/input/video-metrics/818.mp4'
output_video_path = '/kaggle/working/818_enhanced_video.mp4'
process_video(input_video_path, output_video_path)


Processing frame 466/466
Processing completed.
Average PSNR: 15.693198669100951
Average SSIM: 0.7577093327601298
Average Sharpness (Original): 88.76996410182387
Average Sharpness (Enhanced): 151.74040644250192
Average NIQE (Original): 1.090023514258419
Average NIQE (Enhanced): 1.3845401687325816


In [12]:
# Example Usage
input_video_path = '/kaggle/input/video-metrics/996_056.mp4'
output_video_path = '/kaggle/working/996_056_enhanced_video.mp4'
process_video(input_video_path, output_video_path)


Processing frame 312/312
Processing completed.
Average PSNR: 18.823146690480964
Average SSIM: 0.7973118162217795
Average Sharpness (Original): 250.9983918761574
Average Sharpness (Enhanced): 415.20000350403865
Average NIQE (Original): 1.346234450409543
Average NIQE (Enhanced): 1.5665912615448951
